# 第7章 整数計画

## 7.1
>ナップサック問題 (7.7) を、整数計画のソルバーで解いてみよ。
>$$
>\begin{align*}
>\max \ &28x_1 + 33 x_2 + 10 x_3 + 45 x_4 + 32 x_5\\
>\mathrm{s.t.} \ &2x_1 + 3 x_2 + x_3 + 5x_4 + 4 x_5 \le 7\\
>&x_j \in \{0, 1\}, \ \ j = 1, \dots, 5.
>\end{align*}
>$$

In [1]:
from mip import Model, BINARY, xsum, maximize

In [2]:
m = Model()
x = [m.add_var(var_type=BINARY) for _ in range(5)]
c = [28, 33, 10, 45, 32]
a = [2, 3, 1, 5, 4]

m.objective = maximize(xsum(c[i]*x[i] for i in range(5)))
m += xsum(a[i]*x[i] for i in range(5)) <= 7

m.optimize()

from IPython.display import clear_output
clear_output()

In [3]:
print("Status:", m.status)
print("Value:", m.objective_value)
print("Solution: x =", [z.x for z in m.vars])

Status: OptimizationStatus.OPTIMAL
Value: 73.0
Solution: x = [1.0, 0.0, 0.0, 1.0, 0.0]
